In [1]:
#Import libraries
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import csv
import re
import pyspark.sql.functions as sqf
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import split
from pyspark.sql.functions import col, expr, when

# Create a SparkSession and SparkContext
sc = SparkSession.builder.master("local[*]").appName('sparkdf').getOrCreate()

In [3]:
df= sc.read.csv('Amazon_Responded_Oct05.csv',header = True,inferSchema="true")
#Showing only 10 members of selected data
df_fetch3=df[['tweet_created_at','user_screen_name','user_id_str']]
df_fetch3.show(10)

+--------------------+----------------+-----------+
|    tweet_created_at|user_screen_name|user_id_str|
+--------------------+----------------+-----------+
|Tue Nov 01 01:57:...|     SeanEPanjab|  143515471|
|Tue Nov 01 02:39:...|      AmazonHelp|   85741735|
|Tue Nov 01 17:14:...|     SeanEPanjab|  143515471|
|Tue Nov 01 17:15:...|     SeanEPanjab|  143515471|
|Tue Nov 01 17:19:...|      AmazonHelp|   85741735|
|Tue Nov 01 17:25:...|      AmazonHelp|   85741735|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|
|Tue Nov 01 18:02:...|      AmazonHelp|   85741735|
|Tue Nov 01 03:51:...|   aakashwangnoo|   71457972|
+--------------------+----------------+-----------+
only showing top 10 rows



In [4]:
# split data with space '' into 5 parts to fetch only date
tweetdatesplit = split(df_fetch3["tweet_created_at"], ' ')
tweetdatayear= df_fetch3.withColumn('Month', tweetdatesplit.getItem(1)).withColumn('Day', tweetdatesplit.getItem(2)).withColumn('Year', tweetdatesplit.getItem(5))
tweetdatayear.show(10)

+--------------------+----------------+-----------+-----+---+----+
|    tweet_created_at|user_screen_name|user_id_str|Month|Day|Year|
+--------------------+----------------+-----------+-----+---+----+
|Tue Nov 01 01:57:...|     SeanEPanjab|  143515471|  Nov| 01|2016|
|Tue Nov 01 02:39:...|      AmazonHelp|   85741735|  Nov| 01|2016|
|Tue Nov 01 17:14:...|     SeanEPanjab|  143515471|  Nov| 01|2016|
|Tue Nov 01 17:15:...|     SeanEPanjab|  143515471|  Nov| 01|2016|
|Tue Nov 01 17:19:...|      AmazonHelp|   85741735|  Nov| 01|2016|
|Tue Nov 01 17:25:...|      AmazonHelp|   85741735|  Nov| 01|2016|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|  Nov| 01|2016|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|  Nov| 01|2016|
|Tue Nov 01 18:02:...|      AmazonHelp|   85741735|  Nov| 01|2016|
|Tue Nov 01 03:51:...|   aakashwangnoo|   71457972|  Nov| 01|2016|
+--------------------+----------------+-----------+-----+---+----+
only showing top 10 rows



In [5]:
dfconcat1 = tweetdatayear.withColumn("tweet_created_at",sqf.concat(sqf.col("Month"),sqf.col("Day"),sqf.col("Year")))
dfconcat1.show(5)

+----------------+----------------+-----------+-----+---+----+
|tweet_created_at|user_screen_name|user_id_str|Month|Day|Year|
+----------------+----------------+-----------+-----+---+----+
|       Nov012016|     SeanEPanjab|  143515471|  Nov| 01|2016|
|       Nov012016|      AmazonHelp|   85741735|  Nov| 01|2016|
|       Nov012016|     SeanEPanjab|  143515471|  Nov| 01|2016|
|       Nov012016|     SeanEPanjab|  143515471|  Nov| 01|2016|
|       Nov012016|      AmazonHelp|   85741735|  Nov| 01|2016|
+----------------+----------------+-----------+-----+---+----+
only showing top 5 rows



Add spaces to the concatenated field.

In [6]:
dfconcat2 = dfconcat1.withColumn("tweet_created_at",sqf.concat(sqf.col("Month"),sqf.lit(" "),sqf.col("Day"),sqf.lit(" "),sqf.col("Year")))
dfconcat2.show(5)

+----------------+----------------+-----------+-----+---+----+
|tweet_created_at|user_screen_name|user_id_str|Month|Day|Year|
+----------------+----------------+-----------+-----+---+----+
|     Nov 01 2016|     SeanEPanjab|  143515471|  Nov| 01|2016|
|     Nov 01 2016|      AmazonHelp|   85741735|  Nov| 01|2016|
|     Nov 01 2016|     SeanEPanjab|  143515471|  Nov| 01|2016|
|     Nov 01 2016|     SeanEPanjab|  143515471|  Nov| 01|2016|
|     Nov 01 2016|      AmazonHelp|   85741735|  Nov| 01|2016|
+----------------+----------------+-----------+-----+---+----+
only showing top 5 rows



In [7]:
data_final= dfconcat2.select('tweet_created_at','user_screen_name','user_id_str').dropDuplicates()
data_final.show(10)

+----------------+----------------+-----------+
|tweet_created_at|user_screen_name|user_id_str|
+----------------+----------------+-----------+
|     Nov 01 2016|         YIF_JAK|   24787056|
|     Nov 02 2016|     SPWseventy7| 1675908295|
|     Nov 02 2016|  MW3GAMEUPDATES|   40573844|
|     Nov 03 2016| StrongenoughMW1|  391582041|
|     Nov 04 2016|    Linked2Steve|  144895933|
|     Nov 08 2016|     _GAMER_4421| 2504948270|
|     Nov 08 2016|  butchrosenberg| 1536789132|
|     Nov 08 2016|      EuricaeriS|   46162630|
|     Nov 11 2016|   Dogearedtatty|  716850865|
|     Nov 12 2016| HakimUjjainwala|  281911333|
+----------------+----------------+-----------+
only showing top 10 rows



**Step 1:** Find out the users who are active in at least five listed days (i.e., created posts in at least 5 days) in Amazon_Responded_Oct05.csv and save their “user_screen_name” and “user_id_str” in the dataframe “daily_active_users” (see below). Report how many active users you find.

In [8]:
#active users who have created posts in at least 5 days
active_users = data_final.groupBy(data_final.user_screen_name,data_final.user_id_str).count().filter("count >= 5")
daily_active_users = active_users.select('user_screen_name','user_id_str')

#fetch the count
print("COUNT OF DAILY ACTIVE USERS:", daily_active_users.count())
daily_active_users.show(10)

COUNT OF DAILY ACTIVE USERS: 599
+----------------+------------------+
|user_screen_name|       user_id_str|
+----------------+------------------+
|       SkullyRox|          20391647|
|  sky_regenrated|         483059773|
|   Gentlemen_Sam|         441572163|
|        remakoul|814372928695521280|
|          MtnrMS|        3309102108|
|  whisperandmoan|         113516042|
| roxyunderwood93|         295334669|
|        TCMuffin|          35591749|
|  ChaurasiaRohin|706032993794527232|
|        trallyus|          11702402|
+----------------+------------------+
only showing top 10 rows



**Step 2:** A company would like to conduct an A/B test on Twitter. The experiment.txt file includes the user_id_str they selected as potential experiment targets. Please create a dataframe “experiment_user” to document the selected user id and whether they are active users (join the dataframe from step 1).

Then calculate the percentage of active user and print out the result.

In [9]:
#### Read experiments.txt file 

#df_exp = open('experiments.txt', 'r')
#print(exp_df.readline(10))

df_experi= sc.read.csv('experiments.txt',header = False,inferSchema="True")
df_experi = df_experi.withColumnRenamed("_c0", "user_id_experi")
df_experi.show(10)

+--------------+
|user_id_experi|
+--------------+
|     143515471|
|      85741735|
|      71457972|
|    2908108256|
|     106799492|
|      59156981|
|     902137872|
|     110354554|
|      97424433|
|      62364020|
+--------------+
only showing top 10 rows



In [10]:
# join experiments.txt file with data frame daily_active_users.
df_join1 = df_experi.join(daily_active_users,daily_active_users.user_id_str == df_experi.user_id_experi,"left")
df_join1.show(10)

+--------------+----------------+-----------+
|user_id_experi|user_screen_name|user_id_str|
+--------------+----------------+-----------+
|      85741735|      AmazonHelp|   85741735|
|     143515471|            null|       null|
|     110354554| praveen_pandey_|  110354554|
|      97424433|            null|       null|
|     902137872|     mybharatraj|  902137872|
|    2706101936|            null|       null|
|      62364020|            null|       null|
|     106799492|            null|       null|
|    2908108256|            null|       null|
|      71457972|            null|       null|
+--------------+----------------+-----------+
only showing top 10 rows



In [11]:
## Condition to check active users.
whether_active = expr("""If(user_id_str = user_id_experi, "Yes", "No")""")

## creating a new df with Active - Yes or not Active - No status
experiment_user = df_join1.withColumn("whether_active", whether_active).select('user_id_experi','whether_active')
experiment_user.show(10)

## Count fetch for total, active and non active users.
Total_exp_Users = experiment_user.count()
Active = experiment_user.groupBy("whether_active").count().collect()[1]["count"]
Non_Active = experiment_user.groupBy("whether_active").count().collect()[0]["count"]
Percent = Active/(Total_exp_Users)*100

## print results.
print("Total number of Experiment users:", Total_exp_Users)
print("Number of active users are:",Active)
print("Number of non-active users are:",Non_Active)
print("Percentage of active users are:", Percent,"%")

+--------------+--------------+
|user_id_experi|whether_active|
+--------------+--------------+
|     143515471|            No|
|      85741735|           Yes|
|      71457972|            No|
|    2908108256|            No|
|     106799492|            No|
|      59156981|            No|
|     902137872|           Yes|
|     110354554|           Yes|
|      97424433|            No|
|      62364020|            No|
+--------------+--------------+
only showing top 10 rows

Total number of Experiment users: 5000
Number of active users are: 121
Number of non-active users are: 4879
Percentage of active users are: 2.42 %


**Step 3:** In homework 2, you have already known how to join 2 tables in spark. Now you are going to perform a 3-table join task. 
The company provided their revised experiment target list in final_experiment.csv file. Compared with the former experiment.txt file, they removed several users and added a new column “info” to indicate whether the user is female (F) or male (M). However, they are still missing some information. 


In [12]:
#Loading the final_experiment.csv and showing the blank file
final_experiment = sc.read.csv('final_experiment.csv', header = True)
final_experiment.show(10)

+-----------+----+--------------+----------------+
|user_id_str|info|whether_active|user_screen_name|
+-----------+----+--------------+----------------+
|   62364020|   F|          null|            null|
| 2706101936|   M|          null|            null|
|    5654472|   F|          null|            null|
|  145579921|   F|          null|            null|
| 2502172122|   M|          null|            null|
|  243716471|   F|          null|            null|
| 2610379644|   M|          null|            null|
|  123138418|   M|          null|            null|
|  257376764|   F|          null|            null|
|  269145593|   M|          null|            null|
+-----------+----+--------------+----------------+
only showing top 10 rows



In [13]:
# Join aliasing
final_exp=final_experiment.alias('final_exp')
exp_user = experiment_user.alias('exp_user')
active_user = daily_active_users.alias('active_user')

Here we will perfom join for 3 data sources

In [14]:
final_exp = final_exp.drop('whether_active','user_screen_name')

# join experiment and active users
join_actexp = exp_user.join(active_user, active_user.user_id_str == exp_user.user_id_experi, how='left').drop('user_id_str')

## final join
join_final_output = final_exp.join(join_actexp, final_exp.user_id_str == join_actexp.user_id_experi,how='left').drop('user_id_experi')
join_final_output.show(10)

+-----------+----+--------------+----------------+
|user_id_str|info|whether_active|user_screen_name|
+-----------+----+--------------+----------------+
|    5654472|   F|            No|            null|
|   62364020|   F|            No|            null|
|  123138418|   M|            No|            null|
|  145579921|   F|            No|            null|
|  243716471|   F|            No|            null|
|  257376764|   F|            No|            null|
|  269145593|   M|            No|            null|
|  370711133|   F|            No|            null|
| 2502172122|   M|            No|            null|
| 2610379644|   M|            No|            null|
+-----------+----+--------------+----------------+
only showing top 10 rows



In [15]:
final_experiment = join_final_output.na.fill("Not Found")
final_experiment.show(10)

#Getting_count
Yes_count = final_experiment.groupBy("whether_active").count().collect()[2]["count"]
No_Count = final_experiment.groupBy("whether_active").count().collect()[1]["count"]
Not_Found = final_experiment.groupBy("whether_active").count().collect()[0]["count"]

#Printing results.
print("Total Number of users: ",final_experiment.count())
print("Total Number of Not found users",Not_Found)
print("In final_experiment, number of inactive users are:",No_Count)
print("In final_experiment, number of active users are:",Yes_count)

+-----------+----+--------------+----------------+
|user_id_str|info|whether_active|user_screen_name|
+-----------+----+--------------+----------------+
|    5654472|   F|            No|       Not Found|
|   62364020|   F|            No|       Not Found|
|  123138418|   M|            No|       Not Found|
|  145579921|   F|            No|       Not Found|
|  243716471|   F|            No|       Not Found|
|  257376764|   F|            No|       Not Found|
|  269145593|   M|            No|       Not Found|
|  370711133|   F|            No|       Not Found|
| 2502172122|   M|            No|       Not Found|
| 2610379644|   M|            No|       Not Found|
+-----------+----+--------------+----------------+
only showing top 10 rows

Total Number of users:  4500
Total Number of Not found users 262
In final_experiment, number of inactive users are: 4142
In final_experiment, number of active users are: 96


In [16]:
#Convert sparks to pandas dataframes
daily_active_users = daily_active_users.toPandas()
experiment_user = experiment_user.toPandas()
final_experiment = final_experiment.toPandas()

In [19]:
#Export to .csv files
daily_active_users.to_csv('daily_active_users.csv', index = False, header=True)
experiment_user.to_csv('experiment_user.csv', index = False, header=True)
final_experiment.to_csv('final_experiment.csv', index = False, header=True)

Part 2 :
In all 2 joins were used.
1. For the Join condition LEFT join was used under the condition daily_active_users.user_id_str == df_experi.user_id_experi
which helped to join data related to active users and experiment users.
2. For the joining output from Step 1 and Step 2 with Final experiment tables also LEFT join was used again based on user id condition which helped to identify inactive users and were marked as Not found